Downloading Data:
=================

- Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
- Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5

Links:
- https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process/Examples/S2L2A.html#true-color
- https://documentation.dataspace.copernicus.eu/notebook-samples/sentinelhub/cloudless_process_api.html



In [1]:
# Print basic timestamp and python version information

import time
import sys

print("Timestamp: ", time.strftime("%Y-%m-%d %H:%M:%S"))
print("Python version: ", sys.version)

Timestamp:  2025-01-30 10:17:53
Python version:  3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)]


In [2]:
# Update pip itself
!pip install --upgrade pip

# Setup the environment with pip installs and imports
installs = ["sentinelhub", "numpy", "pandas", "matplotlib", "requests", "getpass", "scipy"]
for package in installs:
    !pip install {package}

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------  1.8/1.8 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 7.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
c:\Users\steadmand\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for getpass



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Utilities
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)


In [3]:
# Your client credentials
#Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
#Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5
client_id = 'sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9'
client_secret = 'BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5'

In [4]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

def SetupOAuth(client_id, client_secret):

    # Create a session
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)

    # Get token for the session
    token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                            client_secret=client_secret, include_client_id=True)

    # All requests using this session will have an access token automatically added
    resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")

    return oauth

oauth = SetupOAuth(client_id, client_secret)


In [5]:
# Create strings to support the query

def CreateBBoxString(minlon, minlat, maxlon, maxlat):
    return f"[{minlon}, {minlat}, {maxlon}, {maxlat}]"

def CreateTimerangeString(date):
    # add buffer around the stated date
    datespace = datetime.timedelta(days=2)
    startdate = date - datespace
    enddate = date + datespace

    # format individual date strings
    from_date = f"{startdate.strftime('%Y-%m-%d')}T00:00:00Z"
    to_date = f"{enddate.strftime('%Y-%m-%d')}T23:59:59Z"

    # return the collated/formatted answer
    return {"from": from_date, "to": to_date}



In [6]:
from PIL import Image
from io import BytesIO


def download_GM(outpath, bl_lat, bl_lon, width):

    minlat = bl_lat
    maxlat = bl_lat + width
    minlon = bl_lon
    maxlon = bl_lon + width

    evalscript_GM = """
    //VERSION=3
    function setup() {
        return {
            input: ["B02", "B03", "B04"],
            output: { bands: 3 }
        };
    }

    function evaluatePixel(sample) {
        return [3.5 * sample.B04/10000, 3.5 * sample.B03/10000, 3.5 * sample.B02/10000];
    }
    """

    request_GM = {
    "input": {
        "bounds": {
        "bbox": [minlon, minlat, maxlon, maxlat]
        },
        "data": [
        {
            "dataFilter": {
            "timeRange": {
                "from": "2023-07-01T00:00:00Z",
                "to": "2023-07-01T23:59:59Z"
            }
            },
            "type": "byoc-5460de54-082e-473a-b6ea-d5cbe3c17cca"

        }
        ]
    },
    "output": {
        "width": 1024,
        "height": 1024,
        "responses": [
        {
            "identifier": "default",
            "format": {
                "type": "image/png"
            }
        }
        ]
    },
    "evalscript": evalscript_GM,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    response = oauth.post(url, json=request_GM)

    # Output the result to a file
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image.save(outpath)
    else:
        print(f"Request failed with status code {response.status_code}")



In [2]:
globalmosaic_dates = {
    "2023Q1": "2023-01-01",
    "2023Q2": "2023-04-01",
    "2023Q3": "2023-07-01",
    "2023Q4": "2023-10-01",
    "2024Q1": "2024-01-01",
}

In [1]:
from PIL import Image
from io import BytesIO


def download_GM2(outpath, datestr, bl_lat, bl_lon, width):

    minlat = bl_lat
    maxlat = bl_lat + width
    minlon = bl_lon
    maxlon = bl_lon + width

    # datestr = "2023-07-01"
    fromstr = f"{datestr}T00:00:00Z"
    tostr   = f"{datestr}T23:59:59Z"

    evalscript_GM = """
    //VERSION=3
    function setup() {
        return {
            input: ["B02", "B03", "B04"],
            output: { bands: 3 }
        };
    }

    function evaluatePixel(sample) {
        return [3.5 * sample.B04/10000, 3.5 * sample.B03/10000, 3.5 * sample.B02/10000];
    }
    """

    request_GM = {
    "input": {
        "bounds": {
        "bbox": [minlon, minlat, maxlon, maxlat]
        },
        "data": [
        {
            "dataFilter": {
            "timeRange": {
                "from": fromstr,
                "to": tostr
            }
            },
            "type": "byoc-5460de54-082e-473a-b6ea-d5cbe3c17cca"

        }
        ]
    },
    "output": {
        "width": 1024,
        "height": 1024,
        "responses": [
        {
            "identifier": "default",
            "format": {
                "type": "image/png"
            }
        }
        ]
    },
    "evalscript": evalscript_GM,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    response = oauth.post(url, json=request_GM)

    # Output the result to a file
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image.save(outpath)
    else:
        print(f"Request failed with status code {response.status_code}")



In [7]:
import os

# function to make sure a path exists. If not, create it
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [8]:
# Data structure for the metadata
image_data_5Deg = [
    {"filename": "Sat_EL_DD.png", "lat": -50, "long": 165, "width": 5},
    {"filename": "Sat_EL_CD.png", "lat": -45, "long": 165, "width": 5},
    {"filename": "Sat_EL_DE.png", "lat": -50, "long": 170, "width": 5},
]

image_data_1Deg = [
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_CC.png", "lat": -48, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_CD.png", "lat": -48, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BB.png", "lat": -47, "long": 166, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BC.png", "lat": -47, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BD.png", "lat": -47, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_BE.png", "lat": -47, "long": 169, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AB.png", "lat": -46, "long": 166, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AC.png", "lat": -46, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AD.png", "lat": -46, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DD/", "filename": "Sat_EL_DD_AE.png", "lat": -46, "long": 169, "width": 1},

    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_EC.png", "lat": -45, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_ED.png", "lat": -45, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_EE.png", "lat": -45, "long": 169, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_DC.png", "lat": -44, "long": 167, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_DD.png", "lat": -44, "long": 168, "width": 1},
    {"localpath": "Lvl2_1x1/EL_CD/", "filename": "Sat_EL_CD_DE.png", "lat": -44, "long": 169, "width": 1},

    {"localpath": "Lvl2_1x1/EL_DE/", "filename": "Sat_EL_DE_AA.png", "lat": -46, "long": 170, "width": 1},
    {"localpath": "Lvl2_1x1/EL_DE/", "filename": "Sat_EL_DE_BA.png", "lat": -47, "long": 170, "width": 1},
]


iceland_data_1Deg = [
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_BC.png", "lat": 63, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_BD.png", "lat": 63, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_BE.png", "lat": 63, "long": -21, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AA.png", "lat": 64, "long": -25, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AB.png", "lat": 64, "long": -24, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AC.png", "lat": 64, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AD.png", "lat": 64, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FB/", "filename": "Sat_AF_FB_AE.png", "lat": 64, "long": -21, "width": 1},

    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EA.png", "lat": 65, "long": -25, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EB.png", "lat": 65, "long": -24, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EC.png", "lat": 65, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_ED.png", "lat": 65, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_EE.png", "lat": 65, "long": -21, "width": 1},

    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DA.png", "lat": 66, "long": -25, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DB.png", "lat": 66, "long": -24, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DC.png", "lat": 66, "long": -23, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DD.png", "lat": 66, "long": -22, "width": 1},
    {"localpath": "Lvl2_1x1/AF_EB/", "filename": "Sat_AF_EB_DE.png", "lat": 66, "long": -21, "width": 1},

    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AA.png", "lat": 64, "long": -20, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AB.png", "lat": 64, "long": -19, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AC.png", "lat": 64, "long": -18, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AD.png", "lat": 64, "long": -17, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_AE.png", "lat": 64, "long": -16, "width": 1},

    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BA.png", "lat": 63, "long": -20, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BB.png", "lat": 63, "long": -19, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BC.png", "lat": 63, "long": -18, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BD.png", "lat": 63, "long": -17, "width": 1},
    {"localpath": "Lvl2_1x1/AF_FC/", "filename": "Sat_AF_FC_BE.png", "lat": 63, "long": -16, "width": 1},


]

In [9]:
australia_data_5Deg = [
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_DE.png", "lat": -20, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_DF.png", "lat": -20, "long": 115, "width": 5},

    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_EE.png", "lat": -25, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_EF.png", "lat": -25, "long": 115, "width": 5},

    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_FE.png", "lat": -30, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/DJ/", "filename": "Sat_DJ_FF.png", "lat": -30, "long": 115, "width": 5},


    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BA.png", "lat": -10, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BB.png", "lat": -10, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BC.png", "lat": -10, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BD.png", "lat": -10, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BE.png", "lat": -10, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_BF.png", "lat": -10, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CA.png", "lat": -15, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CB.png", "lat": -15, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CC.png", "lat": -15, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CD.png", "lat": -15, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CE.png", "lat": -15, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_CF.png", "lat": -15, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DA.png", "lat": -20, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DB.png", "lat": -20, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DC.png", "lat": -20, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DD.png", "lat": -20, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DE.png", "lat": -20, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_DF.png", "lat": -20, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EA.png", "lat": -25, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EB.png", "lat": -25, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EC.png", "lat": -25, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_ED.png", "lat": -25, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EE.png", "lat": -25, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_EF.png", "lat": -25, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FA.png", "lat": -30, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FB.png", "lat": -30, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FC.png", "lat": -30, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FD.png", "lat": -30, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FE.png", "lat": -30, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/DK/", "filename": "Sat_DK_FF.png", "lat": -30, "long": 145, "width": 5},


    {"localpath": "Lvl1_5x5/DL/", "filename": "Sat_DL_EA.png", "lat": -25, "long": 150, "width": 5},
    {"localpath": "Lvl1_5x5/DL/", "filename": "Sat_DL_FA.png", "lat": -30, "long": 150, "width": 5},

    {"localpath": "Lvl1_5x5/EJ/", "filename": "Sat_EJ_AE.png", "lat": -35, "long": 110, "width": 5},
    {"localpath": "Lvl1_5x5/EJ/", "filename": "Sat_EJ_AF.png", "lat": -35, "long": 115, "width": 5},

    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AA.png", "lat": -35, "long": 120, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AB.png", "lat": -35, "long": 125, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AC.png", "lat": -35, "long": 130, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AD.png", "lat": -35, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AE.png", "lat": -35, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_AF.png", "lat": -35, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_BD.png", "lat": -40, "long": 135, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_BE.png", "lat": -40, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_BF.png", "lat": -40, "long": 145, "width": 5},

    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_CE.png", "lat": -45, "long": 140, "width": 5},
    {"localpath": "Lvl1_5x5/EK/", "filename": "Sat_EK_CF.png", "lat": -45, "long": 145, "width": 5},


    {"localpath": "Lvl1_5x5/EL/", "filename": "Sat_EL_AA.png", "lat": -35, "long": 150, "width": 5},
    {"localpath": "Lvl1_5x5/EL/", "filename": "Sat_EL_BA.png", "lat": -40, "long": 150, "width": 5},


]

In [10]:

iberia_data_5Deg = [
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_DE.png", "lat": 40, "long": -10, "width": 5},
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_DF.png", "lat": 40, "long":  -5, "width": 5},
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_EE.png", "lat": 35, "long": -10, "width": 5},
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_EF.png", "lat": 35, "long":  -5, "width": 5},
]

iberia_data_1Deg = [
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AA.png", "lat": 44, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AB.png", "lat": 44, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AC.png", "lat": 44, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AD.png", "lat": 44, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_AE.png", "lat": 44, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BA.png", "lat": 43, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BB.png", "lat": 43, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BC.png", "lat": 43, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BD.png", "lat": 43, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_BE.png", "lat": 43, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CA.png", "lat": 42, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CB.png", "lat": 42, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CC.png", "lat": 42, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CD.png", "lat": 42, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_CE.png", "lat": 42, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DA.png", "lat": 41, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DB.png", "lat": 41, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DC.png", "lat": 41, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DD.png", "lat": 41, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_DE.png", "lat": 41, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EA.png", "lat": 40, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EB.png", "lat": 40, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EC.png", "lat": 40, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_ED.png", "lat": 40, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DE/", "filename": "Sat_BF_DE_EE.png", "lat": 40, "long":  -6, "width": 1},



    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_AA.png", "lat": 44, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_AB.png", "lat": 44, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_AC.png", "lat": 44, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_AD.png", "lat": 44, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_AE.png", "lat": 44, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_BA.png", "lat": 43, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_BB.png", "lat": 43, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_BC.png", "lat": 43, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_BD.png", "lat": 43, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_BE.png", "lat": 43, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_CA.png", "lat": 42, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_CB.png", "lat": 42, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_CC.png", "lat": 42, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_CD.png", "lat": 42, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_CE.png", "lat": 42, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_DA.png", "lat": 41, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_DB.png", "lat": 41, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_DC.png", "lat": 41, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_DD.png", "lat": 41, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_DE.png", "lat": 41, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_EA.png", "lat": 40, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_EB.png", "lat": 40, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_EC.png", "lat": 40, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_ED.png", "lat": 40, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_DF/", "filename": "Sat_BF_DF_EE.png", "lat": 40, "long":  -1, "width": 1},



    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_AA.png", "lat": 39, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_AB.png", "lat": 39, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_AC.png", "lat": 39, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_AD.png", "lat": 39, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_AE.png", "lat": 39, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_BA.png", "lat": 38, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_BB.png", "lat": 38, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_BC.png", "lat": 38, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_BD.png", "lat": 38, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_BE.png", "lat": 38, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_CA.png", "lat": 37, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_CB.png", "lat": 37, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_CC.png", "lat": 37, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_CD.png", "lat": 37, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_CE.png", "lat": 37, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_DA.png", "lat": 36, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_DB.png", "lat": 36, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_DC.png", "lat": 36, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_DD.png", "lat": 36, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_DE.png", "lat": 36, "long":  -6, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_EA.png", "lat": 35, "long": -10, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_EB.png", "lat": 35, "long":  -9, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_EC.png", "lat": 35, "long":  -8, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_ED.png", "lat": 35, "long":  -7, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EE/", "filename": "Sat_BF_EE_EE.png", "lat": 35, "long":  -6, "width": 1},



    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_AA.png", "lat": 39, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_AB.png", "lat": 39, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_AC.png", "lat": 39, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_AD.png", "lat": 39, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_AE.png", "lat": 39, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_BA.png", "lat": 38, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_BB.png", "lat": 38, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_BC.png", "lat": 38, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_BD.png", "lat": 38, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_BE.png", "lat": 38, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_CA.png", "lat": 37, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_CB.png", "lat": 37, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_CC.png", "lat": 37, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_CD.png", "lat": 37, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_CE.png", "lat": 37, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_DA.png", "lat": 36, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_DB.png", "lat": 36, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_DC.png", "lat": 36, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_DD.png", "lat": 36, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_DE.png", "lat": 36, "long":  -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_EA.png", "lat": 35, "long":  -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_EB.png", "lat": 35, "long":  -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_EC.png", "lat": 35, "long":  -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_ED.png", "lat": 35, "long":  -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_EF/", "filename": "Sat_BF_EF_EE.png", "lat": 35, "long":  -1, "width": 1},

]

In [11]:
westmed_data_5Deg = [
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_DA.png", "lat": 40, "long":   0, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_DB.png", "lat": 40, "long":   5, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_EA.png", "lat": 35, "long":   0, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_EB.png", "lat": 35, "long":   5, "width": 5},
]

westmed_data_1Deg = [
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_AA.png", "lat": 44, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_AB.png", "lat": 44, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_AC.png", "lat": 44, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_AD.png", "lat": 44, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_AE.png", "lat": 44, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_BA.png", "lat": 43, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_BB.png", "lat": 43, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_BC.png", "lat": 43, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_BD.png", "lat": 43, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_BE.png", "lat": 43, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_CA.png", "lat": 42, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_CB.png", "lat": 42, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_CC.png", "lat": 42, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_CD.png", "lat": 42, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_CE.png", "lat": 42, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_DA.png", "lat": 41, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_DB.png", "lat": 41, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_DC.png", "lat": 41, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_DD.png", "lat": 41, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_DE.png", "lat": 41, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_EA.png", "lat": 40, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_EB.png", "lat": 40, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_EC.png", "lat": 40, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_ED.png", "lat": 40, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_DA/", "filename": "Sat_BG_DA_EE.png", "lat": 40, "long": 4, "width": 1},

]


In [12]:
france_data_5Deg = [
    {"localpath": "Lvl1_5x5/BF/", "filename": "Sat_BF_CF.png", "lat": 45, "long": -5, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_CA.png", "lat": 45, "long":  0, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_CB.png", "lat": 45, "long":  5, "width": 5},
]


france_data_1Deg = [
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_AA.png", "lat": 49, "long": -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_AB.png", "lat": 49, "long": -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_AC.png", "lat": 49, "long": -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_AD.png", "lat": 49, "long": -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_AE.png", "lat": 49, "long": -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_BA.png", "lat": 48, "long": -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_BB.png", "lat": 48, "long": -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_BC.png", "lat": 48, "long": -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_BD.png", "lat": 48, "long": -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_BE.png", "lat": 48, "long": -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_CA.png", "lat": 47, "long": -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_CB.png", "lat": 47, "long": -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_CC.png", "lat": 47, "long": -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_CD.png", "lat": 47, "long": -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_CE.png", "lat": 47, "long": -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_DA.png", "lat": 46, "long": -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_DB.png", "lat": 46, "long": -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_DC.png", "lat": 46, "long": -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_DD.png", "lat": 46, "long": -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_DE.png", "lat": 46, "long": -1, "width": 1},

    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_EA.png", "lat": 45, "long": -5, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_EB.png", "lat": 45, "long": -4, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_EC.png", "lat": 45, "long": -3, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_ED.png", "lat": 45, "long": -2, "width": 1},
    {"localpath": "Lvl2_1x1/BF_CF/", "filename": "Sat_BF_CF_EE.png", "lat": 45, "long": -1, "width": 1},



    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_AA.png", "lat": 49, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_AB.png", "lat": 49, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_AC.png", "lat": 49, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_AD.png", "lat": 49, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_AE.png", "lat": 49, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_BA.png", "lat": 48, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_BB.png", "lat": 48, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_BC.png", "lat": 48, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_BD.png", "lat": 48, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_BE.png", "lat": 48, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_CA.png", "lat": 47, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_CB.png", "lat": 47, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_CC.png", "lat": 47, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_CD.png", "lat": 47, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_CE.png", "lat": 47, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_DA.png", "lat": 46, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_DB.png", "lat": 46, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_DC.png", "lat": 46, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_DD.png", "lat": 46, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_DE.png", "lat": 46, "long": 4, "width": 1},

    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_EA.png", "lat": 45, "long": 0, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_EB.png", "lat": 45, "long": 1, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_EC.png", "lat": 45, "long": 2, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_ED.png", "lat": 45, "long": 3, "width": 1},
    {"localpath": "Lvl2_1x1/BG_CA/", "filename": "Sat_BG_CA_EE.png", "lat": 45, "long": 4, "width": 1},


]

In [13]:
belgium_data_5Deg = [
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_BA.png", "lat": 50, "long": 0, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_BB.png", "lat": 50, "long": 5, "width": 5},
]


centralmed_data_5deg = [
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_DC.png", "lat": 40, "long": 10, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_DD.png", "lat": 40, "long": 15, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_DE.png", "lat": 40, "long": 20, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_DF.png", "lat": 40, "long": 25, "width": 5},

    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_EC.png", "lat": 35, "long": 10, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_ED.png", "lat": 35, "long": 15, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_EE.png", "lat": 35, "long": 20, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_EF.png", "lat": 35, "long": 25, "width": 5},

    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_FC.png", "lat": 30, "long": 10, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_FD.png", "lat": 30, "long": 15, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_FE.png", "lat": 30, "long": 20, "width": 5},
    {"localpath": "Lvl1_5x5/BG/", "filename": "Sat_BG_FF.png", "lat": 30, "long": 25, "width": 5},
]


In [14]:
def create_1deg_dictionary(path, filenamestart, startlat, startlong):
    dictionary = {}

    # Loop over 5 lats
    for currlatid in range(5):
        currlatletter = chr(ord('A') + currlatid)  # Get letter A-E for lat
        currlat = startlat + (4 - currlatid)

        # Loop over 5 longs
        for currlongid in range(5):
            currlongletter = chr(ord('A') + currlongid)  # Get letter A-E for long
            currlong = startlong + currlongid

            # Generate the filename based on letters
            currfilename = f"{filenamestart}{currlatletter}{currlongletter}.png"

            # Create dictionary entry
            dictionary[currfilename] = {
                "localpath": path,
                "filename": currfilename,
                "lat": currlat,
                "long": currlong,
                "width": 1  # Assuming each 1-degree tile has a width of 1
            }

    return dictionary

In [15]:
temp_data = create_1deg_dictionary("Lvl2_1x1/BG_BA/", "Sat_BG_BA_", 54, 0)

# print temp_data verbose/formatted
for key, value in temp_data.items():
    print(f"\"{key}\": {value},")

"Sat_BG_BA_AA.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_AA.png', 'lat': 58, 'long': 0, 'width': 1},
"Sat_BG_BA_AB.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_AB.png', 'lat': 58, 'long': 1, 'width': 1},
"Sat_BG_BA_AC.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_AC.png', 'lat': 58, 'long': 2, 'width': 1},
"Sat_BG_BA_AD.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_AD.png', 'lat': 58, 'long': 3, 'width': 1},
"Sat_BG_BA_AE.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_AE.png', 'lat': 58, 'long': 4, 'width': 1},
"Sat_BG_BA_BA.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_BA.png', 'lat': 57, 'long': 0, 'width': 1},
"Sat_BG_BA_BB.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_BB.png', 'lat': 57, 'long': 1, 'width': 1},
"Sat_BG_BA_BC.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_BC.png', 'lat': 57, 'long': 2, 'width': 1},
"Sat_BG_BA_BD.png": {'localpath': 'Lvl2_

In [3]:
def create_sub1deg_dictionary(path, filenamestart, startlat, startlong):
    dictionary = {}

    # Loop over 5 lats
    for currlatid in range(5):
        currlatletter = chr(ord('A') + currlatid)  # Get letter A-E for lat
        currlat = startlat + (4 - currlatid)

        # Loop over 5 longs
        for currlongid in range(5):
            currlongletter = chr(ord('A') + currlongid)  # Get letter A-E for long
            currlong = startlong + currlongid

            # Generate the filename based on letters
            currfilename = f"{filenamestart}{currlatletter}{currlongletter}.png"

            # Create dictionary entry
            dictionary[currfilename] = {
                "localpath": path,
                "filename": currfilename,
                "lat": currlat,
                "long": currlong,
                "width": 1  # Assuming each 1-degree tile has a width of 1
            }

    return dictionary

In [4]:
temp_data = create_sub1deg_dictionary("Lvl2_1x1/BF_BF_CA/", "Sat_BF_BF_CA_", 52, -5)

# print temp_data verbose/formatted
for key, value in temp_data.items():
    print(f"\"{key}\": {value},")

"Sat_BF_BF_CA_AA.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_AA.png', 'lat': 56, 'long': -5, 'width': 1},
"Sat_BF_BF_CA_AB.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_AB.png', 'lat': 56, 'long': -4, 'width': 1},
"Sat_BF_BF_CA_AC.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_AC.png', 'lat': 56, 'long': -3, 'width': 1},
"Sat_BF_BF_CA_AD.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_AD.png', 'lat': 56, 'long': -2, 'width': 1},
"Sat_BF_BF_CA_AE.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_AE.png', 'lat': 56, 'long': -1, 'width': 1},
"Sat_BF_BF_CA_BA.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_BA.png', 'lat': 55, 'long': -5, 'width': 1},
"Sat_BF_BF_CA_BB.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_BB.png', 'lat': 55, 'long': -4, 'width': 1},
"Sat_BF_BF_CA_BC.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_BC.pn

In [26]:
belgium_data_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_BA/", "Sat_BG_BA_", 50, 0)

BG_BB_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_BB/", "Sat_BG_BB_", 50, 5)
BG_CB_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_CB/", "Sat_BG_CB_", 45, 5)


BG_AB_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_AB/", "Sat_BG_AB_", 55,  5)
BG_AC_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_AC/", "Sat_BG_AC_", 55, 10)
BG_AD_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_AD/", "Sat_BG_AD_", 55, 15)
BG_AE_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_AE/", "Sat_BG_AE_", 55, 20)
BG_AF_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_AF/", "Sat_BG_AF_", 55, 25)

BG_BB_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_BB/", "Sat_BG_BB_", 50,  5)
BG_BC_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_BC/", "Sat_BG_BC_", 50, 10)
BG_BD_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_BD/", "Sat_BG_BD_", 50, 15)
BG_BE_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_BE/", "Sat_BG_BE_", 50, 20)
BG_BF_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_BF/", "Sat_BG_BF_", 50, 25)

BG_CC_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_CC/", "Sat_BG_CC_", 45, 10)
BG_CD_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_CD/", "Sat_BG_CD_", 45, 15)
BG_CE_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_CE/", "Sat_BG_CE_", 45, 20)
BG_CF_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_CF/", "Sat_BG_CF_", 45, 25)



AG_FA_1Deg = create_1deg_dictionary("Lvl2_1x1/AG_FA/", "Sat_AG_FA_", 60,  0)
AG_FB_1Deg = create_1deg_dictionary("Lvl2_1x1/AG_FB/", "Sat_AG_FB_", 60,  5)
AG_FC_1Deg = create_1deg_dictionary("Lvl2_1x1/AG_FC/", "Sat_AG_FC_", 60, 10)
AG_FD_1Deg = create_1deg_dictionary("Lvl2_1x1/AG_FD/", "Sat_AG_FD_", 60, 15)
AG_FE_1Deg = create_1deg_dictionary("Lvl2_1x1/AG_FE/", "Sat_AG_FE_", 60, 20)
AG_FF_1Deg = create_1deg_dictionary("Lvl2_1x1/AG_FF/", "Sat_AG_FF_", 60, 25)

# a list of the dictionaries
list_1Degs = [
    AG_FA_1Deg,
    AG_FB_1Deg,
    AG_FC_1Deg,
    AG_FD_1Deg,
    AG_FE_1Deg,
    AG_FF_1Deg,
]


AG_EC_1Deg = create_1deg_dictionary("Lvl2_1x1/AG_EC/", "Sat_AG_EC_", 65, 10)
AG_ED_1Deg = create_1deg_dictionary("Lvl2_1x1/AG_ED/", "Sat_AG_ED_", 65, 15)
AG_EE_1Deg = create_1deg_dictionary("Lvl2_1x1/AG_EE/", "Sat_AG_EE_", 65, 20)
AG_EF_1Deg = create_1deg_dictionary("Lvl2_1x1/AG_EF/", "Sat_AG_EF_", 65, 25)

# a list of the dictionaries
list_1Degs = [
    AG_EC_1Deg,
    AG_ED_1Deg,
    AG_EE_1Deg,
    AG_EF_1Deg,
]



BF_AE_1Deg = create_1deg_dictionary("Lvl2_1x1/BF_AE/", "Sat_BF_AE_", 55, -10)
BF_AF_1Deg = create_1deg_dictionary("Lvl2_1x1/BF_AF/", "Sat_BF_AF_", 55,  -5)
BF_BE_1Deg = create_1deg_dictionary("Lvl2_1x1/BF_BE/", "Sat_BF_BE_", 50, -10)
BF_BF_1Deg = create_1deg_dictionary("Lvl2_1x1/BF_BF/", "Sat_BF_BF_", 50,  -5)


list_1Degs = [
    BF_AE_1Deg,
    BF_AF_1Deg,
    BF_BE_1Deg,
    BF_BF_1Deg,
]

BG_AA_1Deg = create_1deg_dictionary("Lvl2_1x1/BG_AA/", "Sat_BG_AA_", 55, 0)
BF_CE_1Deg = create_1deg_dictionary("Lvl2_1x1/BF_CE/", "Sat_BF_CE_", 45, -10)
BF_BD_1Deg = create_1deg_dictionary("Lvl2_1x1/BF_BD/", "Sat_BF_BD_", 50, -15)

AF_FE_1Deg = create_1deg_dictionary("Lvl2_1x1/AF_FE/", "Sat_AF_FE_", 60, -10)
AF_FF_1Deg = create_1deg_dictionary("Lvl2_1x1/AF_FF/", "Sat_AF_FF_", 60, -5)

list_1Degs = [
    BG_AA_1Deg,
    BF_CE_1Deg,
    BF_BD_1Deg,
    AF_FE_1Deg,
    AF_FF_1Deg
]



# print temp_data verbose/formatted
for key, value in belgium_data_1Deg.items():
    print(f"\"{key}\": {value},")

"Sat_BG_BA_AA.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_AA.png', 'lat': 54, 'long': 0, 'width': 1},
"Sat_BG_BA_AB.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_AB.png', 'lat': 54, 'long': 1, 'width': 1},
"Sat_BG_BA_AC.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_AC.png', 'lat': 54, 'long': 2, 'width': 1},
"Sat_BG_BA_AD.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_AD.png', 'lat': 54, 'long': 3, 'width': 1},
"Sat_BG_BA_AE.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_AE.png', 'lat': 54, 'long': 4, 'width': 1},
"Sat_BG_BA_BA.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_BA.png', 'lat': 53, 'long': 0, 'width': 1},
"Sat_BG_BA_BB.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_BB.png', 'lat': 53, 'long': 1, 'width': 1},
"Sat_BG_BA_BC.png": {'localpath': 'Lvl2_1x1/BG_BA/', 'filename': 'Sat_BG_BA_BC.png', 'lat': 53, 'long': 2, 'width': 1},
"Sat_BG_BA_BD.png": {'localpath': 'Lvl2_

In [29]:
# BC - North America - Central

BC_AA_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_AA/", "Sat_BC_AA_", 55, -120)
BC_AB_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_AB/", "Sat_BC_AB_", 55, -115)
BC_AC_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_AC/", "Sat_BC_AC_", 55, -110)
BC_AD_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_AD/", "Sat_BC_AD_", 55, -105)
BC_AE_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_AE/", "Sat_BC_AE_", 55, -100)
BC_AF_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_AF/", "Sat_BC_AF_", 55,  -95)

BC_BA_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_BA/", "Sat_BC_BA_", 50, -120)
BC_BB_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_BB/", "Sat_BC_BB_", 50, -115)
BC_BC_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_BC/", "Sat_BC_BC_", 50, -110)
BC_BD_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_BD/", "Sat_BC_BD_", 50, -105)
BC_BE_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_BE/", "Sat_BC_BE_", 50, -100)
BC_BF_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_BF/", "Sat_BC_BF_", 50,  -95)

BC_CA_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_CA/", "Sat_BC_CA_", 45, -120)
BC_CB_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_CB/", "Sat_BC_CB_", 45, -115)
BC_CC_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_CC/", "Sat_BC_CC_", 45, -110)
BC_CD_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_CD/", "Sat_BC_CD_", 45, -105)
BC_CE_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_CE/", "Sat_BC_CE_", 45, -100)
BC_CF_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_CF/", "Sat_BC_CF_", 45,  -95)

BC_DA_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_DA/", "Sat_BC_DA_", 40, -120)
BC_DB_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_DB/", "Sat_BC_DB_", 40, -115)
BC_DC_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_DC/", "Sat_BC_DC_", 40, -110)
BC_DD_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_DD/", "Sat_BC_DD_", 40, -105)
BC_DE_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_DE/", "Sat_BC_DE_", 40, -100)
BC_DF_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_DF/", "Sat_BC_DF_", 40,  -95)

BC_EA_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_EA/", "Sat_BC_EA_", 35, -120)
BC_EB_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_EB/", "Sat_BC_EB_", 35, -115)
BC_EC_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_EC/", "Sat_BC_EC_", 35, -110)
BC_ED_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_ED/", "Sat_BC_ED_", 35, -105)
BC_EE_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_EE/", "Sat_BC_EE_", 35, -100)
BC_EF_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_EF/", "Sat_BC_EF_", 35,  -95)

BC_FA_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_FA/", "Sat_BC_FA_", 30, -120)
BC_FB_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_FB/", "Sat_BC_FB_", 30, -115)
BC_FC_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_FC/", "Sat_BC_FC_", 30, -110)
BC_FD_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_FD/", "Sat_BC_FD_", 30, -105)
BC_FE_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_FE/", "Sat_BC_FE_", 30, -100)
BC_FF_1Deg = create_1deg_dictionary("Lvl2_1x1/BC_FF/", "Sat_BC_FF_", 30,  -95)

list_1Degs = [
    BC_AA_1Deg, BC_AB_1Deg, BC_AC_1Deg, BC_AD_1Deg, BC_AE_1Deg, BC_AF_1Deg,
    BC_BA_1Deg, BC_BB_1Deg, BC_BC_1Deg, BC_BD_1Deg, BC_BE_1Deg, BC_BF_1Deg,
    BC_CA_1Deg, BC_CB_1Deg, BC_CC_1Deg, BC_CD_1Deg, BC_CE_1Deg, BC_CF_1Deg,
    BC_DA_1Deg, BC_DB_1Deg, BC_DC_1Deg, BC_DD_1Deg, BC_DE_1Deg, BC_DF_1Deg,
    BC_EA_1Deg, BC_EB_1Deg, BC_EC_1Deg, BC_ED_1Deg, BC_EE_1Deg, BC_EF_1Deg,
    BC_FA_1Deg, BC_FB_1Deg, BC_FC_1Deg, BC_FD_1Deg, BC_FE_1Deg, BC_FF_1Deg,
]


In [ ]:
# BD - North America - East

BD_AA_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_AA/", "Sat_BD_AA_", 55, -90)
BD_AB_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_AB/", "Sat_BD_AB_", 55, -85)
BD_AC_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_AC/", "Sat_BD_AC_", 55, -80)
BD_AD_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_AD/", "Sat_BD_AD_", 55, -75)
BD_AE_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_AE/", "Sat_BD_AE_", 55, -70)
BD_AF_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_AF/", "Sat_BD_AF_", 55, -65)

BD_BA_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_BA/", "Sat_BD_BA_", 50, -90)
BD_BB_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_BB/", "Sat_BD_BB_", 50, -85)
BD_BC_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_BC/", "Sat_BD_BC_", 50, -80)
BD_BD_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_BD/", "Sat_BD_BD_", 50, -75)
BD_BE_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_BE/", "Sat_BD_BE_", 50, -70)
BD_BF_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_BF/", "Sat_BD_BF_", 50, -65)

BD_CA_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_CA/", "Sat_BD_CA_", 45, -90)
BD_CB_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_CB/", "Sat_BD_CB_", 45, -85)
BD_CC_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_CC/", "Sat_BD_CC_", 45, -80)
BD_CD_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_CD/", "Sat_BD_CD_", 45, -75)
BD_CE_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_CE/", "Sat_BD_CE_", 45, -70)
BD_CF_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_CF/", "Sat_BD_CF_", 45, -65)

BD_DA_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_DA/", "Sat_BD_DA_", 40, -90)
BD_DB_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_DB/", "Sat_BD_DB_", 40, -85)
BD_DC_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_DC/", "Sat_BD_DC_", 40, -80)
BD_DD_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_DD/", "Sat_BD_DD_", 40, -75)
BD_DE_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_DE/", "Sat_BD_DE_", 40, -70)
BD_DF_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_DF/", "Sat_BD_DF_", 40, -65)

BD_EA_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_EA/", "Sat_BD_EA_", 35, -90)
BD_EB_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_EB/", "Sat_BD_EB_", 35, -85)
BD_EC_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_EC/", "Sat_BD_EC_", 35, -80)
BD_ED_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_ED/", "Sat_BD_ED_", 35, -75)
BD_EE_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_EE/", "Sat_BD_EE_", 35, -70)
BD_EF_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_EF/", "Sat_BD_EF_", 35, -65)

BD_FA_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_FA/", "Sat_BD_FA_", 30, -90)
BD_FB_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_FB/", "Sat_BD_FB_", 30, -85)
BD_FC_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_FC/", "Sat_BD_FC_", 30, -80)
BD_FD_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_FD/", "Sat_BD_FD_", 30, -75)
BD_FE_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_FE/", "Sat_BD_FE_", 30, -70)
BD_FF_1Deg = create_1deg_dictionary("Lvl2_1x1/BD_FF/", "Sat_BD_FF_", 30, -65)

list_1Degs = [
    BC_AA_1Deg, BC_AB_1Deg, BC_AC_1Deg, BC_AD_1Deg, BC_AE_1Deg, BC_AF_1Deg,
    BC_BA_1Deg, BC_BB_1Deg, BC_BC_1Deg, BC_BD_1Deg, BC_BE_1Deg, BC_BF_1Deg,
    BC_CA_1Deg, BC_CB_1Deg, BC_CC_1Deg, BC_CD_1Deg, BC_CE_1Deg, BC_CF_1Deg,
    BC_DA_1Deg, BC_DB_1Deg, BC_DC_1Deg, BC_DD_1Deg, BC_DE_1Deg, BC_DF_1Deg,
    BC_EA_1Deg, BC_EB_1Deg, BC_EC_1Deg, BC_ED_1Deg,
    BC_FA_1Deg, BC_FB_1Deg, BC_FC_1Deg,
]


In [ ]:
# CC - Mexico

CC_AA_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_AA/", "Sat_CC_AA_", 25, -120)
CC_AB_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_AB/", "Sat_CC_AB_", 25, -115)
CC_AC_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_AC/", "Sat_CC_AC_", 25, -110)
CC_AD_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_AD/", "Sat_CC_AD_", 25, -105)
CC_AE_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_AE/", "Sat_CC_AE_", 25, -100)
CC_AF_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_AF/", "Sat_CC_AF_", 25,  -95)

CC_BA_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_BA/", "Sat_CC_BA_", 20, -120)
CC_BB_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_BB/", "Sat_CC_BB_", 20, -115)
CC_BC_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_BC/", "Sat_CC_BC_", 20, -110)
CC_BD_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_BD/", "Sat_CC_BD_", 20, -105)
CC_BE_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_BE/", "Sat_CC_BE_", 20, -100)
CC_BF_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_BF/", "Sat_CC_BF_", 20,  -95)

CC_CA_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_CA/", "Sat_CC_CA_", 15, -120)
CC_CB_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_CB/", "Sat_CC_CB_", 15, -115)
CC_CC_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_CC/", "Sat_CC_CC_", 15, -110)
CC_CD_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_CD/", "Sat_CC_CD_", 15, -105)
CC_CE_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_CE/", "Sat_CC_CE_", 15, -100)
CC_CF_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_CF/", "Sat_CC_CF_", 15,  -95)

CC_DA_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_DA/", "Sat_CC_DA_", 10, -120)
CC_DB_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_DB/", "Sat_CC_DB_", 10, -115)
CC_DC_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_DC/", "Sat_CC_DC_", 10, -110)
CC_DD_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_DD/", "Sat_CC_DD_", 10, -105)
CC_DE_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_DE/", "Sat_CC_DE_", 10, -100)
CC_DF_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_DF/", "Sat_CC_DF_", 10,  -95)

CC_EA_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_EA/", "Sat_CC_EA_",  5, -120)
CC_EB_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_EB/", "Sat_CC_EB_",  5, -115)
CC_EC_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_EC/", "Sat_CC_EC_",  5, -110)
CC_ED_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_ED/", "Sat_CC_ED_",  5, -105)
CC_EE_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_EE/", "Sat_CC_EE_",  5, -100)
CC_EF_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_EF/", "Sat_CC_EF_",  5,  -95)

CC_FA_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_FA/", "Sat_CC_FA_",  0, -120)
CC_FB_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_FB/", "Sat_CC_FB_",  0, -115)
CC_FC_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_FC/", "Sat_CC_FC_",  0, -110)
CC_FD_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_FD/", "Sat_CC_FD_",  0, -105)
CC_FE_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_FE/", "Sat_CC_FE_",  0, -100)
CC_FF_1Deg = create_1deg_dictionary("Lvl2_1x1/CC_FF/", "Sat_CC_FF_",  0,  -95)

list_1Degs = [
    BC_AA_1Deg, BC_AB_1Deg, BC_AC_1Deg, BC_AD_1Deg, BC_AE_1Deg, BC_AF_1Deg,
                BC_BB_1Deg, BC_BC_1Deg, BC_BD_1Deg, BC_BE_1Deg, BC_BF_1Deg,
                            BC_CC_1Deg, BC_CD_1Deg, BC_CE_1Deg, BC_CF_1Deg,
                                        BC_DD_1Deg, BC_DE_1Deg, BC_DF_1Deg,

]

In [ ]:
# BD - Cuba

CD_AA_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_AA/", "Sat_CD_AA_", 25, -90)
CD_AB_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_AB/", "Sat_CD_AB_", 25, -85)
CD_AC_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_AC/", "Sat_CD_AC_", 25, -80)
CD_AD_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_AD/", "Sat_CD_AD_", 25, -75)
CD_AE_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_AE/", "Sat_CD_AE_", 25, -70)
CD_AF_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_AF/", "Sat_CD_AF_", 25, -65)

CD_BA_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_BA/", "Sat_CD_BA_", 20, -90)
CD_BB_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_BB/", "Sat_CD_BB_", 20, -85)
CD_BC_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_BC/", "Sat_CD_BC_", 20, -80)
CD_BD_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_BD/", "Sat_CD_BD_", 20, -75)
CD_BE_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_BE/", "Sat_CD_BE_", 20, -70)
CD_BF_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_BF/", "Sat_CD_BF_", 20, -65)

CD_CA_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_CA/", "Sat_CD_CA_", 15, -90)
CD_CB_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_CB/", "Sat_CD_CB_", 15, -85)
CD_CC_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_CC/", "Sat_CD_CC_", 15, -80)
CD_CD_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_CD/", "Sat_CD_CD_", 15, -75)
CD_CE_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_CE/", "Sat_CD_CE_", 15, -70)
CD_CF_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_CF/", "Sat_CD_CF_", 15, -65)

CD_DA_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_DA/", "Sat_CD_DA_", 20, -90)
CD_DB_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_DB/", "Sat_CD_DB_", 20, -85)
CD_DC_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_DC/", "Sat_CD_DC_", 20, -80)
CD_DD_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_DD/", "Sat_CD_DD_", 20, -75)
CD_DE_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_DE/", "Sat_CD_DE_", 20, -70)
CD_DF_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_DF/", "Sat_CD_DF_", 20, -65)

CD_EA_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_EA/", "Sat_CD_EA_",  5, -90)
CD_EB_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_EB/", "Sat_CD_EB_",  5, -85)
CD_EC_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_EC/", "Sat_CD_EC_",  5, -80)
CD_ED_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_ED/", "Sat_CD_ED_",  5, -75)
CD_EE_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_EE/", "Sat_CD_EE_",  5, -70)
CD_EF_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_EF/", "Sat_CD_EF_",  5, -65)

CD_FA_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_FA/", "Sat_CD_FA_",  0, -90)
CD_FB_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_FB/", "Sat_CD_FB_",  0, -85)
CD_FC_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_FC/", "Sat_CD_FC_",  0, -80)
CD_FD_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_FD/", "Sat_CD_FD_",  0, -75)
CD_FE_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_FE/", "Sat_CD_FE_",  0, -70)
CD_FF_1Deg = create_1deg_dictionary("Lvl2_1x1/CD_FF/", "Sat_CD_FF_",  0, -65)

list_1Degs = [
    CD_AA_1Deg, CD_AB_1Deg, CD_AC_1Deg,
    CD_BA_1Deg, CD_BB_1Deg, CD_BC_1Deg, CD_BD_1Deg,
    CD_CA_1Deg, CD_CB_1Deg, CD_CC_1Deg, CD_CD_1Deg, CD_CE_1Deg, CD_CF_1Deg,
    CD_DA_1Deg, CD_DB_1Deg, CD_DC_1Deg, CD_DD_1Deg, CD_DE_1Deg, CD_DF_1Deg,
    CD_EA_1Deg, CD_EB_1Deg, CD_EC_1Deg, CD_ED_1Deg, CD_EE_1Deg, CD_EF_1Deg,
                            CD_FC_1Deg, CD_FD_1Deg, CD_FE_1Deg, CD_FF_1Deg,
]


In [17]:
RootDir = "C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/"
ensure_dir(RootDir)

brightness = 35

oauth = SetupOAuth(client_id, client_secret)


# Loop over the data structure
for data in belgium_data_1Deg:

    TilesetDir = os.path.join(RootDir, data["localpath"])
    filepath   = os.path.join(TilesetDir, data["filename"])

    ensure_dir(TilesetDir)

    print(filepath)

    if not os.path.exists(filepath):

        print(f"Downloading and enhancing image: {data['filename']}")

        # Call your functions using the metadata
        download_GM(filepath, data["lat"], data["long"], data["width"])
        # change_absolute_brightness(filepath, filepath, brightness)
        #adjust_image_properties(filepath, filepath, brightness_delta=30, contrast_factor=1.5, saturation_factor=1.5)





TypeError: string indices must be integers, not 'str'

In [18]:
RootDir = "C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/"
ensure_dir(RootDir)

brightness = 35

oauth = SetupOAuth(client_id, client_secret)


# Loop over the data structure
for data in BG_CF_1Deg.values():
    TilesetDir = os.path.join(RootDir, data["localpath"])
    filepath   = os.path.join(TilesetDir, data["filename"])

    ensure_dir(TilesetDir)

    print(filepath)

    if not os.path.exists(filepath):

        print(f"Downloading and enhancing image: {data['filename']}")

        # Call your functions using the metadata
        download_GM(filepath, data["lat"], data["long"], data["width"])
        # change_absolute_brightness(filepath, filepath, brightness)
        #adjust_image_properties(filepath, filepath, brightness_delta=30, contrast_factor=1.5, saturation_factor=1.5)



C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_AA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_AB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_AC.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_AD.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_AE.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_BA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_BB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_BC.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_BD.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_BE.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_CA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_CB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_CF/Sat_BG_CF_CC.png

In [20]:
RootDir = "C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/"
ensure_dir(RootDir)
brightness = 35

def process_tileset_dict(tileset_dict):

    oauth = SetupOAuth(client_id, client_secret)

    for data in tileset_dict.values():
        TilesetDir = os.path.join(root_dir, data["localpath"])
        filepath   = os.path.join(TilesetDir, data["filename"])

        ensure_dir(TilesetDir)

        print(filepath)

        if not os.path.exists(filepath):

            print(f"Downloading and enhancing image: {data['filename']}")

            # Call your functions using the metadata
            download_GM(filepath, data["lat"], data["long"], data["width"])


In [27]:
# Loop over the dictionaries and call the download function for each

root_dir = "C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/"

oauth = SetupOAuth(client_id, client_secret)

for tileset_dict in list_1Degs:
    process_tileset_dict(tileset_dict)

C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_AA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_AB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_AC.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_AD.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_AE.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_BA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_BB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_BC.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_BD.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_BE.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_CA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_CB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl2_1x1/BG_AA/Sat_BG_AA_CC.png